In [ ]:
import gymnasium as gym
import numpy as np

env = gym.make('CartPole-v1')

goal_average_steps = 195
max_number_of_steps = 200
num_consecutive_iterations = 100
num_episodes = 200
last_time_steps = np.zeros(num_consecutive_iterations)

for episode in range(num_episodes):
    # 環境の初期化
    observation, info = env.reset()

    episode_reward = 0
    for t in range(max_number_of_steps):
        # CartPoleの描画
        env.render()
        if env.viewer == None:
            env.render()
        # ランダムで行動の選択
        action = np.random.choice([0, 1])

        # 行動の実行とフィードバックの取得
        observation, reward, terminate, truncated, info = env.step(action)
        episode_reward += reward

        if terminate or truncated:
            print('%d Episode finished after %d time steps / mean %f' % (episode, t + 1,
                last_time_steps.mean()))
            last_time_steps = np.hstack((last_time_steps[1:], [episode_reward]))
            break

    if (last_time_steps.mean() >= goal_average_steps): # 直近の100エピソードが195以上であれば成功
        print('Episode %d train agent successfuly!' % episode)
        break

0 Episode finished after 40 time steps / mean 0.000000
1 Episode finished after 22 time steps / mean 0.400000
2 Episode finished after 15 time steps / mean 0.620000
3 Episode finished after 25 time steps / mean 0.770000
4 Episode finished after 19 time steps / mean 1.020000
5 Episode finished after 27 time steps / mean 1.210000
6 Episode finished after 25 time steps / mean 1.480000
7 Episode finished after 48 time steps / mean 1.730000
8 Episode finished after 10 time steps / mean 2.210000
9 Episode finished after 25 time steps / mean 2.310000
10 Episode finished after 26 time steps / mean 2.560000
11 Episode finished after 11 time steps / mean 2.820000
12 Episode finished after 9 time steps / mean 2.930000
13 Episode finished after 13 time steps / mean 3.020000
14 Episode finished after 10 time steps / mean 3.150000
15 Episode finished after 11 time steps / mean 3.250000
16 Episode finished after 19 time steps / mean 3.360000
17 Episode finished after 13 time steps / mean 3.550000
18 

In [1]:
import gymnasium as gym
import numpy as np

env = gym.make('CartPole-v1')

In [2]:
import numpy as np
q_table = np.random.uniform(low=-1, high=1, size=(4 ** 4, env.action_space.n))

def bins(clip_min, clip_max, num):
    return np.linspace(clip_min, clip_max, num + 1)[1:-1]

def digitize_state(observation):
    # 各値を4個の離散値に変換
    cart_pos, cart_v, pole_angle, pole_v = observation
    digitized = [np.digitize(cart_pos, bins=bins(-2.4, 2.4, 4)),
                 np.digitize(cart_v, bins=bins(-3.0, 3.0, 4)),
                 np.digitize(pole_angle, bins=bins(-0.5, 0.5, 4)),
                 np.digitize(pole_v, bins=bins(-2.0, 2.0, 4))]
    # 0~255に変換
    return sum([x * (4 ** i) for i, x in enumerate(digitized)])

In [13]:
q_table


array([[ 4.81051356e-01,  2.47133814e-01],
       [ 2.84290843e-01, -4.32111516e-01],
       [ 6.59550867e-01,  1.23622011e-01],
       [-1.17757614e-01, -2.04252908e-01],
       [ 6.05512444e-01, -7.32100112e-01],
       [-1.39107777e-01, -9.93657621e-01],
       [ 8.89884623e-04, -2.14839905e-01],
       [ 7.83216384e-01,  2.90776821e-01],
       [ 2.90441223e-01,  6.26698722e-01],
       [ 7.30157167e-01,  3.73401080e-01],
       [ 2.13991355e-01, -1.23487344e-01],
       [ 1.15235194e-01,  3.60374842e-01],
       [-4.32411681e-01,  2.04736985e+01],
       [ 4.80565105e+01, -4.33680346e-01],
       [-7.90959558e-01,  2.28182825e-01],
       [ 2.00800573e+01, -3.42237035e-01],
       [-3.00564513e-01,  1.86169660e-01],
       [ 5.51079041e-01, -8.21742593e-02],
       [ 5.57343708e-01,  6.04287110e-02],
       [ 3.26440707e-01,  7.44040760e-01],
       [ 9.25185230e+01, -8.41804452e-01],
       [-2.28323535e-01, -2.95139656e-01],
       [ 7.32275232e-01, -1.13177504e-02],
       [ 1.

In [4]:
env.action_space.n

2

In [9]:
goal_average_steps = 195
max_number_of_steps = 500
num_consecutive_iterations = 100
num_episodes = 2000
last_time_steps = np.zeros(num_consecutive_iterations)

In [12]:
def get_action(state, action, observation, reward):
    next_state = digitize_state(observation)
    next_action = np.argmax(q_table[next_state])

    # Qテーブルの更新
    alpha = 0.2
    gamma = 0.99
    q_table[state, action] = (1 - alpha) * q_table[state, action] +\
            alpha * (reward + gamma * q_table[next_state, next_action])

    return next_action, next_state

for episode in range(num_episodes):
    # 環境の初期化
    observation, info = env.reset()

    state = digitize_state(observation)
    action = np.argmax(q_table[state])

    episode_reward = 0
    for t in range(max_number_of_steps):
        # CartPoleの描画
        #env.render()

        # 行動の実行とフィードバックの取得
        observation, reward, ter, tran, info = env.step(action)

        # 行動の選択
        action, state = get_action(state, action, observation, reward)
        episode_reward += reward

        if ter or tran:
            break
    print(episode_reward)

14.0
196.0
11.0
14.0
14.0
17.0
26.0
30.0
11.0
148.0
14.0
58.0
120.0
14.0
14.0
14.0
163.0
12.0
10.0
15.0
14.0
11.0
149.0
130.0
93.0
15.0
200.0
155.0
12.0
186.0
15.0
200.0
22.0
17.0
169.0
11.0
21.0
11.0
11.0
141.0
14.0
18.0
10.0
11.0
129.0
200.0
12.0
18.0
11.0
31.0
11.0
200.0
17.0
31.0
29.0
20.0
168.0
12.0
170.0
12.0
10.0
11.0
12.0
12.0
27.0
154.0
200.0
11.0
13.0
24.0
25.0
14.0
11.0
170.0
9.0
160.0
20.0
22.0
14.0
22.0
10.0
14.0
16.0
31.0
144.0
99.0
26.0
15.0
40.0
38.0
105.0
14.0
35.0
11.0
9.0
200.0
27.0
15.0
114.0
158.0
43.0
10.0
11.0
14.0
128.0
130.0
200.0
12.0
200.0
200.0
10.0
15.0
10.0
18.0
137.0
12.0
11.0
200.0
110.0
11.0
114.0
14.0
167.0
156.0
164.0
200.0
22.0
29.0
174.0
12.0
14.0
15.0
13.0
188.0
146.0
27.0
18.0
42.0
24.0
14.0
16.0
167.0
20.0
11.0
18.0
39.0
187.0
60.0
10.0
10.0
34.0
19.0
11.0
24.0
151.0
12.0
16.0
136.0
181.0
15.0
21.0
179.0
24.0
145.0
11.0
193.0
21.0
27.0
137.0
15.0
12.0
125.0
11.0
23.0
13.0
186.0
125.0
200.0
11.0
129.0
11.0
153.0
11.0
200.0
10.0
32.0
22.0
9.0
200.0

In [8]:
q_table[0]

array([0.48105136, 0.24713381])

In [8]:
action, state = get_action(state, action, observation, reward)
print(action)
print(state)

0
9
